# Foundations of Durable AI with Temporal

By now, you've experienced generative AI firsthand. You've used ChatGPT and seen what LLMs can do. They excel at tasks like research, but their real power emerges when we connect them with users and other actions to build more advanced applications that go beyond simple chat interfaces.

In this workshop, we'll build toward creating AI agents, but let's start with a simple chain:

Use an LLM to generate research -> then produce a PDF from that research.

### What are GenAI Applications?

At their core, GenAI applications use an LLM as one component among many. The LLM isn't the application itself. Take ChatGPT as an example—it's an application that wraps an LLM, not an LLM itself. Even this seemingly simple chat interface does much more than just call an LLM:

- Displays responses to the user
- Captures user input
- Maintains conversation history
- Orchestrates each subsequent LLM call

Applications can look like many different formats. We will start with something like a chain workflow where a series of LLM calls, actions, and user interactions are strung together:

<img src="https://images.ctfassets.net/0uuz8ydxyd9p/70SBemKQHnqfLxoHgPovQX/33f3a0b6cfc96eae2d17d1a463079560/Screenshot_2025-07-08_at_10.26.26%C3%A2__AM.png" />

Applications can also be agentic, where the path through the business logic isn't predetermined. AI agents are GenAI applications where the LLM has agency over the functionality and flow of the application.

<img src="https://i.postimg.cc/zv5W9VkH/Screenshot-2025-10-07-at-7-48-20-PM.png" width="300"/>

We'll first look at applications that look like a chain workflow.

### Let's Prompt an LLM

APIs are used to supply an LLM with context and get back a response.

### Hands-on Moments

This is a hands-on workshop!

All of the instructors slides and code samples are are executable in the workshop notebooks.
We encourage you to follow along and play with the samples!

At the end of every chapter (notebook) will be a hands-on lab.
This a self-guided experience where the instructor gives a prompt (not an llm haha) with a notebook and some starter code and the attendees solve the puzzle.

We are going to create a Research Agent that makes a call to the OpenAI API, conducts research on a topic of your choice, and generates a PDF report from that research. Let's go ahead and first set up your notebook.

In [ ]:
# We'll first install the necessary packages for this workshop.

%pip install --quiet litellm reportlab python-dotenv

### Create an `.env` File

Next you'll create a `.env` file to store your API keys.
In the file browser on the left, create a new file and name it `.env`.

**Note**: It may disappear as soon as you create it. This is because Google Collab hides hidden files (files that start with a `.`) by default.
To make this file appear, click the icon that is a crossed out eye and hidden files will appear.

Then double click on the `.env` file and add the following line with your API key.

```
LLM_API_KEY = YOUR_API_KEY
LLM_MODEL = "openai/gpt-4o"
```

By default this notebook uses OpenAI's GPT-4o.
If you want to use a different LLM provider, look up the appropriate model name [in their documentation](https://docs.litellm.ai/docs/providers) and change the `LLM_MODEL` field and provide your API key.

In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

# Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings

import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

### Prompting the LLM

Our agent will use LLM calls to process information and decide what actions to take.

We use `litellm` here, which is a unified interface for over 100+ LLM providers. This means that the same code works with different models - you only need to change the model string. All you need to do is provide an API key.

In [ ]:
# Run this codeblock to load it into the program
from litellm import completion, ModelResponse

def llm_call(prompt: str, llm_api_key: str, llm_model: str) -> ModelResponse:
    response = completion(
      model=llm_model,
      api_key=llm_api_key,
      messages=[{ "content": prompt,"role": "user"}]
    )
    return response

prompt = "Give me 5 facts about elephants."
result = llm_call(prompt, LLM_API_KEY, LLM_MODEL)["choices"][0]["message"]["content"]

print(result)

### Making It Interactive

Great! You've successfully made your first LLM call with a hardcoded prompt. Now, let's transform this into an interactive research assistant that will perform research on any topic of your choosing.

This is the first step toward building our Gen-AI application.

In [ ]:
# Run this codeblock to load it into the program
# Make the API call
print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ")
result = llm_call(prompt, LLM_API_KEY, LLM_MODEL)

# Extract the response content
response_content = result["choices"][0]["message"]["content"]

print("Research complete!")
print("-"*80)
print(response_content)

### Now, let's process the LLM results.

So far, we’ve only returned the response from the LLM to the user.

1. Pass results into a "next step"
2. Add results to conversation history
3. Parse the results

We want to do more.

### From Prompts to Actions

Our LLM can only generate text responses - it thinks and responds, but it can't *do* anything in the real world.

What if we want our LLM to:
- Search the web for the latest information?
- Save the research report to a file?
- Send the results via email?
- Query a database or call an API?

This is where **actions** come in.

### What is an Action?

An action is an external function that performs specific tasks beyond just generating text.

Examples:
- Information retrieval (web search, database query, file reading)
- Communication tools (sending emails, post to Slack, sending text messages or notifications)
- Data analysis tools (run calculations, generate charts and graphs)
- Creative tools (image generation, document creation)

### Building Your First Action

Let's enhance our research application by adding an action that saves the results to a PDF. This will demonstrate how to move from just generating text to performing concrete actions with that text.

### Generating a PDF

Once you have your research data, you’ll call an action that takes the results and writes it out to a PDF.

Remember, actions can interact with the outside world:
- File operations (like this PDF generator)
- API calls to external services
- Database queries
- Email sending
- Web scraping

In [ ]:
## Let's create our PDF generation action.
## This function takes text content and formats it into a professional-looking PDF document:

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

def create_pdf(content: str, filename: str = "research_report.pdf") -> str:
    doc = SimpleDocTemplate(filename, pagesize=letter)

    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))

    paragraphs = content.split('\n\n')
    for para in paragraphs:
        if para.strip():
            p = Paragraph(para.strip(), styles['Normal'])
            story.append(p)
            story.append(Spacer(1, 12))

    doc.build(story)
    return filename

create_pdf("Hello PDF!", filename="test.pdf")

### Open the PDF

Download the `test.pdf` PDF and open it by following these steps:
  - Right-click the PDF file in the file explorer
  - Select "Download"
  - Open it locally on your machine

You should see a title **Research Report** and the words **Hello PDF!** in the document.

### Bringing it all Together

You now have multiple functions that you can execute to achieve a task.
Next, write the code to bring this all together:

1. Use the LLM to respond to a prompt
2. Call the `create_pdf` action to create a PDF with the response to your prompt.

See how neat this is? Instead of just printing text to the console, your application now creates a tangible deliverable. You ask a question, get a response, and walk away with a professional PDF report.

In [ ]:
# Step 1: Call the `llm_call` function with `prompt`, `LLM_API_KEY`, and `LLM_MODEL` as arguments.
# Step 2: Call the `create_pdf` function with `response_content` as the first argument.
# Step 3: Run the code block to execute the program.
# Step 4: # Download the `research_report.pdf` PDF and open it by following these steps:
#  - Right-click the PDF file in the file explorer
#  - Select "Download"
#  - Open it locally on your machine

# Make the API call
print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ")
result = llm_call() # TODO: Call the `llm_call` function with `prompt`, `LLM_API_KEY`, and `LLM_MODEL` as arguments.

# Extract the response content
response_content: str = result["choices"][0]["message"]["content"]

pdf_filename = # Call the `create_pdf` function that takes in our 2 arguments: response_content and "research_report.pdf"
print(f"SUCCESS! PDF created: {pdf_filename}")

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_01_Foundations_of_Durable_AI_with_Temporal" / "pdf_generation_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### The Foundations of a Chain Workflow

You now have the foundations of a chain workflow application. We chained an LLM call (`llm_call`) and action together (`create_pdf`) in a defined order (user input → LLM call → PDF generation).

### Demo (Expand for instructor notes or to run on your own)
<!--
Normal Execution Demo:
1. To demonstrate the power of durable execution, we'll first show the power of running the app with no durable execution. This is the code that we showed in the first notebook.
2. Clone this repository: `https://github.com/temporalio/edu-ai-workshop-agentic-loop`. The instructions will also be in the README.
2. From the `demos/module_one_01_foundations_aiic_loop/app.py` directory, run `app.py` with `python app.py`.
3. When prompted, provide the prompt you want to prompt OpenAI in the command line.
4. Before the process generates a PDF, kill the process.
5. Rerun the application again with `python app.py` and show that the process restarted and you have to have your application start the research again. Emphasize that from a cost perspective, this could be very costly, because you could have to re-run through many tokens to get to where you left off.

Durable Execution Demo:
1. Now show the durable version by switching into the ``demos/module_one_02_adding_durability` directory.
2. Run the Worker with `python worker.py`.
3. Run the Workflow with `python workflow.py`.
4. When prompted, provide the prompt you want to prompt OpenAI in the command line.
5. Before the process generates a PDF, kill the Worker.
6. Rerun the Worker and show that you continue right where you left off.
7. Emphasize that you lost no progress or data. The Workflow will continue by generating the PDF (available in the same directory) and completing the process successfully.
10. Show the Workflow Execution completion in the Web UI.
-->

### Challenges of GenAI Applications

When you combine LLMs with other actions—calling external APIs, querying databases, processing files, you're coordinating multiple services across network boundaries.

But challenges can happen:

- Networks can be flakey
- LLMs are often rate limited
- Tool resources (APIs and databases) go down
- LLMs are inherently non-deterministic
- How do we scale these applications?
- What happens when they take a long time to finish?
…
What else?

**GenAI based applications are distributed systems**. And we are going to show you how to make these are resilient.

<img src="https://i.postimg.cc/xCWJ2qym/chain-workflow-challenges.png" width="400"/>

## Agents are distributed systems!

<img src="https://i.postimg.cc/hj7vpcW4/Screenshot-2025-10-07-at-8-11-37-PM.png" width="400"/>

### The Challenges Only Amplify with Agents

What you built:
- User input → LLM call → PDF generation
- A simple 3-step chain

However, there are a few significant challenges to getting production AI at scale. Your simple agent works great in demos, but production environments are messy and unpredictable.

In production, this becomes:
- **Longer chains**: User input → Web search → LLM analysis → Database query → LLM refinement → PDF generation → Email delivery
- **External dependencies**: APIs, databases, file systems
- **Network failures**: Any step can fail
- **Expensive retries**: Re-running a 30-second LLM call because step 5 failed

Imagine if the user asks for report → LLM times out → half-generated PDF → frustrated user

<img src="https://i.postimg.cc/bJQ8ZJft/plan-observe-execute.png" width="300"/>

### Distributed System Issues

When workflows get even more sophisticated, they evolve into **agentic systems** - where the LLM makes decisions about which actions to take next, potentially calling other specialized agents that themselves have complex workflows.

Increasingly, we are seeing agents calling agents, which are calling other agents.

For example, your research application might:
- Call a "Web Scraper" agent to gather sources (scrapes 5 research websites)
- Call a "Fact Checker" agent to verify claims (validates statistics against government data)
- Call a "PDF Generator" agent (what you built!)

Each agent has its own event loop **(Plan → Execute → Observe)**:

For example: Research Agent → Executes web search → Observes results → Decides if it needs more data → Executes another search or moves to analysis → Observes → Decides next step → etc.

<img src="https://i.postimg.cc/sXRCCrG1/agents-calling-agents.png" width="500"/>

### This means your "simple" research request triggers a complex orchestration.

<img src="https://i.postimg.cc/SKdfkLJW/agent-orchestration.png" width="500"/>

### Now overlay what can go wrong: network partitions, timeouts, and service failures at any step can break the chain anywhere!

<img src="https://i.postimg.cc/9MftM49H/agent-orchestration-problems.png" width="500"/>

The bottom line: **What looks like one AI task is actually a distributed system challenge!**

### Agents are Distributed Systems!

**This is why durability matters.** Without it, complex workflows become fragile and expensive. With it, failures become manageable interruptions instead of catastrophic losses.

### What Does Durability Mean?

- If an LLM call fails halfway through processing, you **don't lose the work already completed**.
- If a database query times out, you can **retry just that step** without restarting everything.
- If your application crashes, it can **resume from the last successful operation**.
- **Long-running processes** can span hours or days without losing context.

Without durability, every failure means starting over.
With durability, failures become recoverable interruptions instead of catastrophic losses. This is especially critical for GenAI applications where LLM calls are expensive, slow, and unpredictable.

### AI Needs Durability

  Your research application needs to:
  1. Accept user input
    - Possible problems: input validation service, rate limiting
  2. Call the LLM for research
    - Possible problems: Internet connection, API down, rate limiting, timeout
  3. Generate PDF
    - Possible problems: Memory limits
  4. Return success/failure
    - Possible problem: Connection dropped

  Each step can fail.
  Each step might need different agents.
  This is a **workflow** - and workflows need orchestration.

---

# Exercise 1 - Adding More Actions

* In this exercise, you'll:
  * Call actions with your GenAI Application
  * Extract structured information from LLM responses to coordinate between different actions.
* Go to the **Exercise** Directory and open the **01_Foundations_of_Durable_AI_with_Temporal** Directory
* Open the _Practice_ directory and follow the instructions and filling in the `TODO` statements.
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers

### What's Next?

This workshop introduced you the importance of durability in GenAI applications. Further your learning with these resources:

### Resources

- [A mental model for agentic AI Applications blogpost](https://temporal.io/blog/a-mental-model-for-agentic-ai-applications)
- [From AI hype to durable reality — why agentic flows need distributed-systems discipline blogpost](https://temporal.io/blog/from-ai-hype-to-durable-reality-why-agentic-flows-need-distributed-systems)